In [ ]:
!pip install langdetect
import pandas as pd
import requests
from bs4 import BeautifulSoup
from langdetect import detect
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlparse


'''import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()'''

#nltk.download('stopwords',download_dir = '/root/nltk_data')
#nltk.download("punkt")

In [ ]:
urldata = pd.read_csv('/content/knu_students.csv')[['KboNr','URL','NACE']].head(1400)
textdata = pd.DataFrame({'KboNr':urldata['KboNr'],'Text':[None] * len(urldata),'NACE':urldata['NACE']})

In [ ]:
def scrape_website(url, original_url = True):
    try:
        response = requests.get(str(url))
        if response.status_code == 200:
            #soup = BeautifulSoup(response.content, 'html.parser')
            soup = BeautifulSoup(response.text, 'html.parser')
            text = soup.get_text()
            text = re.sub(r'\s+', ' ', text).strip()
            return text
        else:
            if original_url:
              add_url_error(url, response.status_code)
              print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error occurred while scraping {url}: {e}")
        return None


def get_domain(url):
    parsed_url = urlparse(url)
    domain = f"{parsed_url.scheme}://{parsed_url.netloc}/"
    return domain

def add_url_error(url, status_code):  #This is to check which errors occur and how often.
                                      #The URL get's added to the list now only for troubleshooting.
    error_urls[status_code].append(url)




def process_url(row, languages=['nl'], min_word_count=20): #Voor engels moeten we ook nog een oplossing vinden om de site naar nederlands te brengen over te slaan.
                                                                  #Engelse text mogen we niet scrapen.
    url = row.URL  # Extract the URL from the row
    url = get_domain(url)
    original_text = scrape_website(url)

    if original_text:
        language = detect(original_text)
        #if language != f'nl':
        if language not in languages:
          if language == f'fr':
            attempt_nr = 0 #attempt_nr to keep track of how many times URL conversion has been attempted
                            # This way we can use multiple methods until one works.
            try_new_url = True #Set to true so we can stop when we find dutch site

            while try_new_url:
              new_url, attempt_nr = convert_url_fr(url, attempt_nr) #Try to convert URL and keep track of attempt number.
              new_text = scrape_website(new_url, False)
              new_language = detect(original_text)

              if new_language == f'nl':
                original_text == new_text
                try_new_url = False

              elif attempt_nr == 0:
                try_new_url = False
                print(f"Language error: {new_language} Failed to retrieve content from {url}")
                return None

          else: #If language is not dutch and not french
              print(f"Language error: {language} Failed to retrieve content from {url}")
              return None

        if len(original_text.split()) >= min_word_count:# and language in languages:
            return original_text

        elif len(original_text.split()) < min_word_count:
                print(f"Text from {url} doesn't meet the word count criteria (actual count: {len(original_text.split())}, required count: {min_word_count})")

        else:
             print(f"Unkown Problem {url}")
             return None
    else:
        print(f"Failed to retrieve content from {url}")
        return None

def convert_url_fr(url, attempt_nr):
  if attempt_nr == 0:
    attempt_nr += 0
    if ('/fr' in url):
        return url.replace('/fr','/nl'), attempt_nr
    if ('/en' in url):
      return url.replace('/en','/nl'), attempt_nr
    if ('/de' in url):
        return url.replace('/de','/nl'), attempt_nr
    elif (url[-1:] == '/'):
        return url + 'nl', attempt_nr
    elif (url[-1:] != '/'):
        return url + '/nl', attempt_nr
    #skip these steps at the moment.
    '''elif attempt_nr == 1: #?lang=fr
    attempt_nr = 0
    if ('lang=fr' in url):
        return url.replace('/lang=fr','/lang=nl'), attempt_nr
    if ('lang=en' in url):
        return url.replace('/lang=en','/lang=nl'), attempt_nr
    if ('lang=de' in url):
        return url.replace('/lang=de','/lang=nl'), attempt_nr
    elif (url[-1:] == '/'):
        return url + '?lang=nl', attempt_nr
    elif (url[-1:] != '/'):
        return url + '?lang=nl', attempt_nr

    elif attempt_nr == 2: #?lang=fr
      attempt_nr = 0'''

    parsed_url = urlparse(url)
    # Construct the new URL with the language code as a subdomain
    new_url = f"{parsed_url.scheme}://nl.{parsed_url.netloc}{parsed_url.path}"
    return new_url, attempt_nr


In [ ]:
def main():
    # Set the maximum number of threads (adjust as needed)
    max_threads = 35
    error_urls = {
      100: [],  101: [],  102: [],
      200: [],  201: [],  202: [],  203: [],  204: [],  205: [],  206: [],
      300: [],  301: [],  302: [],  303: [],  304: [],  305: [],  306: [],  307: [],  308: [],
      400: [],  401: [],  402: [],  403: [],  404: [],  405: [],  406: [],  407: [],  408: [],
      409: [],  410: [],  411: [],  412: [],  413: [],  414: [],  415: [],  416: [],  417: [],
      418: [],  421: [],  422: [],  423: [],  424: [],  425: [],  426: [],  428: [], 429: [],
      431: [], 451: [],
      500: [], 501: [], 502: [], 503: [], 504: [], 505: [], 506: [], 507: [], 508: [],
      510: [], 511: []
      }

    with ThreadPoolExecutor(max_threads) as executor:

        #loop = asyncio.get_event_loop()
        #tasks = [loop.run_in_executor(executor, process_url, url) for url in urldata]
        #tasks = [loop.process_url(url) for url in urldata]
        #results = await asyncio.gather(*tasks)

        results = list(executor.map(process_url, urldata.itertuples(index=False)))


    # Assign values to the corresponding columns in textdata
    for i, result in enumerate(results):
        if result is not None:
            textdata.at[i, 'Text'] = result
        else:
            textdata.at[i, 'Text'] = None

    textdata.to_csv('/content/textdata.csv')

    for key in error_urls.keys():
      if len(error_urls[key]) != 0:
        print(f'Error code {key}: {error_urls[key]}')


    print(textdata['Text'].count())
    print(textdata['KboNr'].count())

In [ ]:
if __name__ == "__main__":
    main()
'''loop = asyncio.get_event_loop()
task = asyncio.ensure_future(main())
loop.run_until_complete(asyncio.sleep(2))'''

Runtime 1375 urls: 6m50-7m10s
Runtime (est) 260.000 urls: 21,5h22,6h
Succes 1375 urls: 673
Succes (est) 260.000 urls: 190.000
Data size 1375 urls: 2,3MB
Data size (est) 260.000 urls: 800MB



                '''elif language not in languages:
                nl_url = convert_url_fr(url)
                nl_text = scrape_website(nl_url)'''

                '''elif nl_text and len(nl_text.split()) >= min_word_count:
                return nl_text, nl_text
                tokens = word_tokenize(nl_text)
                stop_words = set(stopwords.words("dutch"))
                filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

                words_to_be_removed = ["cookie", "cookies", "tracking", "privacy"]
                filtered_tokens = [word for word in filtered_tokens if word.lower() not in words_to_be_removed]

                stemmer = PorterStemmer()
                stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

                text = ' '.join(filtered_tokens)
                stemmed_text = ' '.join(stemmed_tokens)

                return text, stemmed_text
                else:
                    return None, None'''

            '''tokens = word_tokenize(original_text)
            if language == 'en':
                stop_words = set(stopwords.words("english"))
            elif language == "nl":
                stop_words = set(stopwords.words("dutch"))
            filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

            words_to_be_removed = ["cookie", "cookies", "tracking", "privacy"]
            filtered_tokens = [word for word in filtered_tokens if word.lower() not in words_to_be_removed]

            stemmer = PorterStemmer()
            stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

            text = ' '.join(filtered_tokens)
            stemmed_text = ' '.join(stemmed_tokens)

            return text, stemmed_text'''